In [32]:
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import *

In [33]:
#圖片為黑白的故為 1 channel
input_shape = (256, 256, 1)
inputs = Input(shape=input_shape)
#一開始的卷積面積為16
start=16
#MaxPooling跑6層，UpSampling跑6+1層，最多設到6，設到7會報錯
layerCount=6
x = Conv2D(start, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2), padding='same')(x)
for r in range(0,layerCount):
    #卷積面積透過迴圈依2倍放大
    start=start*2
    x = Conv2D(start, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
#6+1層的那個+1層
x = UpSampling2D((2, 2))(x)
x = Conv2D(start, (3, 3), activation='relu', padding='same')(x)
for r in range(0,layerCount):
    #卷積面積透過迴圈依2倍縮小
    start=start/2
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(start, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
model = Model(inputs=inputs, outputs=x)
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [34]:
from data import *

In [35]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
b=1
train = trainGenerator(b,r'img\folder5','train','trainmask',data_gen_args,save_to_dir = None)
val= trainGenerator(b,r'img\folder5','validation','validationmask',data_gen_args,save_to_dir = None)

In [36]:
model_checkpoint = ModelCheckpoint('FCN_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model_checkpoint2=EarlyStopping(monitor='loss', patience=5)
model.fit(train,steps_per_epoch=5000,epochs=30,callbacks=[model_checkpoint,model_checkpoint2],validation_steps=val)

Found 3614 images belonging to 1 classes.
Found 3614 images belonging to 1 classes.
Epoch 1/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0450 - accuracy: 0.9845
Epoch 1: loss improved from inf to 0.04500, saving model to FCN_membrane.hdf5
5000/5000 [==============================] - 130s 26ms/step - loss: 0.0450 - accuracy: 0.9845
Epoch 2/30
4998/5000 [============================>.] - ETA: 0s - loss: 0.0321 - accuracy: 0.9859
Epoch 2: loss improved from 0.04500 to 0.03207, saving model to FCN_membrane.hdf5
5000/5000 [==============================] - 123s 25ms/step - loss: 0.0321 - accuracy: 0.9859
Epoch 3/30
5000/5000 [==============================] - ETA: 0s - loss: 0.0294 - accuracy: 0.9868
Epoch 3: loss improved from 0.03207 to 0.02944, saving model to FCN_membrane.hdf5
5000/5000 [==============================] - 125s 25ms/step - loss: 0.0294 - accuracy: 0.9868
Epoch 4/30
4999/5000 [============================>.] - ETA: 0s - loss: 0.0273 - accuracy: 0.9877


In [37]:
test = testGenerator(r"img\folder5\test")
#有多少張圖
testCount=46
results = model.predict_generator(test,testCount,verbose=1)
saveResult(r"img\folder5\predict",results)

C:\Users\user\AppData\Local\Temp\ipykernel_25136\2930822701.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(test,testCount,verbose=1)


46/46 [==============================] - 1s 11ms/step
